In [ ]:
# !pip install geopy

In [ ]:
# sudo -u postgres nominatim serve  --project-dir /data/nominatim-planet 

In [1]:
# import re
# import json
import pandas as pd
from geopy.geocoders import Nominatim
from tqdm.notebook import tqdm

tqdm.pandas()
pd.options.display.max_colwidth = 130 # увеличим максимальную длину столбцов pandas

In [2]:
!ls data

avito_2022-09-01_14-07_raw.xlsx  avito_2022-09-01_14-07.xlsx  old


In [59]:
ts = '2022-09-01_14-07'

In [3]:
df = pd.read_excel(f'data/avito_{ts}.xlsx')
print(len(df))
df.sample(2)

3953


,adr,obj_name,title,priceM,nrooms,floor,nfloors,area,is_studio,is_apartment,is_part,is_auction,is_openspace,is_SNT,is_last_floor,is_roof,description,price,avito_page,avito_id
3584,"ул. Генерала Мельника, 9",NaN,"2-к. квартира, 56,1 м², 9/9 эт.",9.2,2,9,9,56.1,False,False,False,False,False,False,True,False,"Замечательная квартира с ремонтом, можно наслаждаться пейзажом с двух сторон дома. Лоджия с выходом на кухню и комнату. Тихий ...",9200000,72,2546736589
2337,"проспект Генерала Острякова, 171Б",NaN,"3-к. квартира, 70 м², 4/10 эт.",9.0,3,4,10,70.0,False,False,False,False,False,False,False,False,"Трёхкомнатная квартира с шикарной планировкой. Две большие лоджии, три больших встроенных шкафа. До остановки общественного тр...",9000000,47,2532431314


---

In [4]:
# sudo -u postgres nominatim serve  --project-dir /data/nominatim-planet 

geolocator = Nominatim(user_agent='http://127.0.0.1:8088')

adr = 'севастополь острякова 57'
location = geolocator.geocode(adr)
coo = (location.latitude, location.longitude)
coo

(44.5773585, 33.520378)

In [58]:
def get_coo(adr, city_prefix='Севастополь', geolocator=geolocator):
    if len(city_prefix)>0: 
        adr_ =  city_prefix + ', ' + adr
    adr_ = adr_.split(',')
    loc = geolocator.geocode(adr_)
    
    truncated = False
    while (loc is None) and ( len(adr_) > 2 ): # если не нашел...
        adr_ = adr_[:-1] # выкидываем часть адреса 
        loc = geolocator.geocode( adr_) # ... и пробуем ещё раз
        truncated = True
        
    return (
        {'coo':None,'truncated':None} 
        if (loc is None) 
        else {'coo':(loc.latitude, loc.longitude),'truncated':truncated} 
    )

# location = df[['adr']].drop_duplicates().sample(30).reset_index(drop=True)
    
location = df[['adr']].drop_duplicates().reset_index(drop=True)

location['coo'] = location['adr'].progress_apply(get_coo)
location['is_adr_truncated'] = location['coo'].str['truncated']
location['coo'] = location['coo'].str['coo']

location.sample(4)

  0%|          | 0/1545 [00:00<?, ?it/s]

,adr,coo,is_adr_truncated
1036,"ул. Подольцева, 6","(44.5965395, 33.5325455)",False
208,"проспект Генерала Острякова, 20","(44.547665, 33.5355546)",False
1531,"пос. городского типа Кача, Красноармейская ул., 2",None,None
815,"пос. городского типа Кача, ул. Авиаторов, 1Гк5",None,None


In [63]:
# location.to_excel(f'data/avito_{ts}_location.xlsx', index=False)
location.to_pickle(f'data/avito_{ts}_location.pkl')

----

In [57]:
# geolocator.geocode('Севастополь, Аэродромное шоссе, 2 к34')
# geolocator.geocode('Севастополь, проспект Генерала Острякова, 244 к7')
# geolocator.geocode('Севастополь, ул. Астана Кесаева,')
# geolocator.geocode('Севастополь, ул. Николая Музыки')
# geolocator.geocode('Севастополь, ул. Коли Пищенко, 20А'.split(',')[:-1])
# geolocator.geocode('Севастополь, с.Тыловое, Севастопольская ул., 6')
# geolocator.geocode('Севастополь, с.Тыловое')
# 'Севастополь, ул. Коли Пищенко, 20А'.split(',')[:-1]
# geolocator.geocode('Севастополь, садоводческое некоммерческое товарищество Бриз')
# geolocator.geocode('с. Песчаное, ')

In [ ]:
# geolocator.geocode?

In [ ]:
# import re
# location['adr'].apply(lambda s: re.sub(r'(\d)(к\d)',r'\1 \2',s))

In [ ]:
# geolocator.geocode('Севастополь садоводческое товарищество Кача, 1')

In [ ]:
# !pip install folium

In [ ]:
# import folium

# m = folium.Map(location=coo,zoom_start=23)
# folium.Marker(coo,popup=adr,tooltip=adr).add_to(m)
# m

---

In [ ]:
# !pip install osmnx

In [ ]:
# import osmnx as ox

# center_point = (40.4168, -3.7038)
# G = ox.graph_from_point(center_point, dist=15000, retain_all=True, simplify = True, network_type='all')

---

In [ ]:
# !pip install nominatim

In [ ]:
# from nominatim import Nominatim
# #from nominatim import NominatimReverse

# nom = Nominatim('http://127.0.0.1:8088')

In [ ]:
# nom.query(u'москва', acceptlanguage='RU')
# # nom.query(u'moscow')

In [ ]:
# u'москва'.encode('utf-8').strip()

In [ ]:
# http://127.0.0.1:8088/search.php?q=севастополь

---

In [ ]:
# from ipyleaflet import AwesomeIcon

# # m = Map(center=(55.718148, 37.555493), zoom=14)

In [ ]:
# location['adr_'] = 

# location['coo'] = (
#     (
#     'Севастополь, '
#      +location['adr']
#      .apply(lambda s: re.sub(r'(\d)(к\d)',r'\1 \2',s))
#      .str.replace(' ш.,',' шоссе,')
#      .str.replace('пр-т','проспект')
#     )
#     location['adr_'] 
#     .progress_apply(geolocator.geocode )
#     .apply(lambda x: x if x is None else (x.latitude, x.longitude))
# )

# # ('город Севастополь, '+location['adr']).progress_apply( geolocator.geocode )
# location

# location['coo'] = (
#     ( 'Севастополь, ' + location['adr'] )
#     .progress_apply(geolocator.geocode )
#     .apply(lambda x: x if x is None else (x.latitude, x.longitude))
# )

# ('город Севастополь, '+location['adr']).progress_apply( geolocator.geocode )